Podaci koje gledamo su zadani u train.json i test.json. Stoga prvo moramo učitati  i spremiti podatke iz danih datoteka.

JSON datoteka je datoteka koja pohranjuje jednostavne strukture podataka i objekte u formatu JavaScript Object Notation, koji je standardni format za razmjenu podataka. Najviše se koristi za prijenos podataka između web-aplikacije i poslužitelja. One su temeljene na teksu, lako čitljive i mogu se uređivati pomoću uređivača teksta.

In [1]:
dfs = {}
for name in ['train', 'test']:
    df = pd.read_json('%s.json' % name)
    df['_data'] = name
    dfs[name] = df

Koristeći funkciju append spajamo podatke iz 'train' i 'test' datoteka unutar 'df' i zatim ograničavamo podatke na zajedničke stupce te uključujemo indikator za predviđanje uspjeha zahtjeva.

In [ ]:

df = dfs['train'].append(dfs['test'])
df = df.reset_index(drop=True)


cols = list(dfs['test'].columns) + ['requester_received_pizza']
df = df[cols]


Nakon što smo preimenovali stupce radi preglednosti, pretvaramo varijablu 'got_pizza' u integer, izbacujemo neiskorištene stupce te na kraju spajamo 'title' i 'body' stupce. 

In [ ]:
df.rename(columns={
        'request_title': 'title', 
        'request_text_edit_aware': 'body',
        'requester_received_pizza': 'got_pizza',
}, inplace=True)

df['got_pizza'] = df['got_pizza'].apply(lambda x: -1 if pd.isnull(x) else int(x))

cols_to_keep = ['_data', 'request_id', 'title', 'body', 'got_pizza']
df = df[cols_to_keep]

df.iloc[0]

df['txt_raw'] = df['title'] + ' ' + df['body']

U NLTK-u postoje tzv. 'stopwords' odnosno riječi koje nam ne daju nikakvo značenje o samoj rečenici te izbacivanjem istih ne mijenjamo strukturu i značenje te rečenica. Te riječi učitavamo pomoću naredbe 'from nltk.corpus import stopwords'. 

Uz izbacivanje 'stopwords', također preoblikujemo tekst tako što sva slova pretvaramo u mala slova, te se riješavamo stvari (znakovi i sl.) koje nam nisu od koristi za daljnji rad i rezultat algoritma.



In [ ]:
import re
def clean_txt(raw, remove_stop=False):
    
    letters_only = re.sub("[^a-zA-Z]", " ", raw) 

    words = letters_only.lower().split()                             

    stops = set(stopwords.words("english"))
    words = [w for w in words if not w in stops]
    
    return " ".join(words)

df['txt_clean'] = df['txt_raw'].apply(clean_txt)

Skup podataka ćemo pripremiti za daljnju obradu unutar algoritma tako što ćemo konstruirati numeričko polje koje predstavlja znakove iz teksta transformirane u brojeve radi lakše buduće implementacije i obrade algoritma. 

In [ ]:
def get_xy(vectorizer=None, txt_col='txt_clean'):
    if vectorizer is None:
        vectorizer = CountVectorizer()
        
    dg = df[df['_data'] == 'train']

    X = vectorizer.fit_transform(dg[txt_col]).toarray()
    y = dg['got_pizza'].astype(int).to_numpy()

    return X, y

Nakon nasumične podijele skupa podataka za treniranje isprobavamo naivni Bayesov model.

In [ ]:
X, y = get_xy()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1234)

model = MultinomialNB().fit(X_train, y_train)

print ("Tocnost na skupu podataka za treniranje: %f" % model.score(X_train, y_train))
print ("Tocnost na skupu podataka za testiranje: %f" % model.score(X_test, y_test))

y_pred = model.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
print ("AUC: %f" % auc(fpr, tpr))

Tocnost na skupu podataka za treniranje: 0.885149

Tocnost na skupu podataka za testiranje: 0.717822

AUC: 0.512876

U implementaciji algoritma koristimo dijagnostičke alate 'roc' i 'auc' koji nam pomažu interpretirati vjerojatnosnu prognozu problema klasificiranja prediktivnog modeliranja zbog određivanja polarnosti zahtjeva. Na temelju toga donosimo predviđanje uspješnosti tog istog zahtjeva.

Biramo vrijednosti koje maksimiziraju varijable 'roc' i 'auc' te nakon toga prolazimo kroz petlju varijabli kako bismo pronašli optimalne vrijednosti.

In [ ]:
alphas = [1, 5, 10, 25]
min_dfs = [0.001, 0.01, 0.02, 0.05]

best_alpha, best_min_df = None, None
max_auc = -np.inf

for alpha in alphas:
    for min_df in min_dfs:
        
        vectorizer = CountVectorizer(min_df = min_df)

        X, y = get_xy(vectorizer)
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

        model = MultinomialNB(alpha=alpha).fit(X_train, y_train)

        y_pred = model.predict_proba(X_test)[:, 1]        
        fpr, tpr, thresholds = roc_curve(y_test, y_pred)
        auc_val = auc(fpr, tpr)

        if auc_val > max_auc:
            max_auc = auc_val
            best_alpha, best_min_df = alpha, min_df 

                
print ("alpha: %f" % best_alpha)
print ("min_df: %f" % best_min_df)
print ("best auc: %f" % max_auc)

alpha: 5.000000

min_df: 0.020000

best auc: 0.605254

Provjeravamo je li sada model poboljšan.

In [ ]:
vectorizer = CountVectorizer(min_df = best_min_df)

X, y = get_xy(vectorizer)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1234)

model = MultinomialNB(alpha=best_alpha).fit(X_train, y_train)

print ("Tocnost na skupu podataka za treniranje: %f" % model.score(X_train, y_train))
print ("Tocnost na skupu podataka za testiranje:     %f" % model.score(X_test, y_test))

Tocnost na skupu podataka za treniranje: 0.757756

Tocnost na skupu podataka za testiranje: 0.724752

Treniramo cijeli skup podataka za treniranje sa najboljim parametrima te spremamo predviđanja. Zatim kreiramo csv datoteku.

In [ ]:
vectorizer = CountVectorizer(min_df = best_min_df)

X, y = get_xy(vectorizer)

model = MultinomialNB(alpha=best_alpha).fit(X, y)

df_test = df[df['_data'] == 'test'].copy()
X_test = vectorizer.transform(df_test['txt_clean'])
y_pred = model.predict_proba(X_test)[:, 1]

df_test['requester_received_pizza'] = y_pred
final_df = df_test[['request_id', 'requester_received_pizza']]

final_df.to_csv('sampleSubmission.csv', index=False)

Za kraj pogledajmo koji skup riječi najbolje garantira uspješnost zahtjeva.

In [ ]:
words = np.array(vectorizer.get_feature_names())

x = np.eye(X.shape[1])
probs = model.predict_proba(x)[:, 1]

word_df = pd.DataFrame()
word_df['word'] = words
word_df['P(pizza | word)'] = probs
word_df.sort_values('P(pizza | word)', ascending=False, inplace=True)

print ('good words')
print (word_df.head(10))
print ('\n---\n')
print ('bad words')
print (word_df.tail(10))

In [ ]:
good words
           word  P(pizza | word)
161         jpg         0.427790
268        rice         0.383345
157       imgur         0.364823
325       tight         0.363433
141     helping         0.361993
235      person         0.354807
345  unemployed         0.344983
231    paycheck         0.338030
233      paying         0.337769
50        check         0.336646

---

bad words
         word  P(pizza | word)
34   birthday         0.181946
169     leave         0.180133
104   florida         0.175216
326      till         0.172488
112   friends         0.165682
20       area         0.162599
108      free         0.159392
285   sitting         0.159047
307  studying         0.155792
111    friend         0.143568